In [ ]:
!pip install plotly
!pip install pmdarima

# Analisis de politicas_dia_anterior_Acumulado.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Se puede ver la data del archivo:

In [ ]:
!pip install --upgrade -q pygsheets

In [ ]:
import google.auth
from google.colab import auth

auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
def gsheet2df(file):
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  worksheet = gc.open(file).sheet1

  # get_all_values gives a list of rows.
  rows = worksheet.get_all_values()
  # Convert to a DataFrame and render.
  import pandas as pd
  df=pd.DataFrame.from_records(rows)
  df.columns = df.iloc[0]
  df = df.iloc[1:]
  return df

In [ ]:
import pandas as pd
import numpy as np

dataProbabilidad= gsheet2df('politicas_dia_anterior_Acumulado.csv')
tira = gsheet2df('TIRA.csv').rename(columns={"ATM":"atm"})
dataProbabilidad["fecha"]=pd.to_datetime(dataProbabilidad["fecha"], format='%m/%d/%y', errors='ignore')
dataProbabilidad=dataProbabilidad.merge(tira,on="atm").astype({'atm':'int64','abastecer':'float64','probabilidad_stockout':'float64','EFECTIVO':'int64'})
dataProbabilidad.head()

In [ ]:
ultima_fecha=dataProbabilidad.fecha.max()
print("ultima fecha con la cual se encuentra la data:"+str(ultima_fecha))
data0=dataProbabilidad[(dataProbabilidad.fecha==ultima_fecha)&(dataProbabilidad.abastecer==0)]
data1=dataProbabilidad[(dataProbabilidad.fecha==ultima_fecha)&(dataProbabilidad.abastecer==1)]

In [ ]:
import plotly.graph_objs as go
from IPython.display import HTML
dat=dataProbabilidad.loc[dataProbabilidad.fecha==ultima_fecha,["EFECTIVO","probabilidad_stockout"]]
# Create traces
trace =go.Scatter(x=dat.EFECTIVO,y=dat.probabilidad_stockout, mode='markers')
layout = dict(title = 'Relacion entre Efectivo y probabilidad de desabastecimiento de los cajeros',
              xaxis= dict(title= 'Efectivo',ticklen= 5,zeroline= False,gridwidth= 2),
            yaxis=dict(title= 'Probabilidad de desabastecimiento',ticklen= 5,gridwidth= 2),
            showlegend= False)

fig = go.Figure(data=[trace],layout=layout)
HTML(fig.to_html())

In [ ]:
tiraH = gsheet2df('Tira_Historico_Acumulado.csv')
politicasH = gsheet2df('Politicas_Historico_Acumulado.csv').rename(columns={"atm":"ATM","fecha":"FECHA"})
politicasH["FECHA"]=pd.to_datetime(politicasH["FECHA"], format='%d/%m/%Y', errors='ignore')
tiraH["FECHA"]=pd.to_datetime(tiraH["FECHA"], format='%Y-%m-%d %H:%M:%S', errors='ignore')

Lista de atm que se puede buscar la grafica de tira y probabilidad de desabastecimiento historicos:

In [ ]:
import plotly.graph_objs as go
import plotly.offline as offline
# Create traces
draw=dict()
draw1=dict()
maladata=[]
#solo los 50 atm
politicas50 = gsheet2df('politicas_dia_anterior-17Feb.csv').rename(columns={"atm":"ATM","fecha":"FECHA"})
for atm in politicas50.ATM.unique():
    dat=politicasH[politicasH.ATM==atm].sort_values(by=['FECHA']).reset_index()
    dat1=tiraH[tiraH.ATM==atm].sort_values(by=['FECHA']).reset_index()
    draw[str(atm)]=go.Scatter(x=dat.FECHA, y=dat.probabilidad_stockout,mode='lines+markers',name="atm:"+str(atm))
    draw1[str(atm)]=go.Scatter(x=dat1.FECHA, y=dat1.EFECTIVO,mode='lines+markers',name="atm:"+str(atm))
print(list(set(draw.keys()).intersection(set(draw1.keys()))))  

In [ ]:
idx=input("Ingrese el numero de atm: ")

In [ ]:
from IPython.display import HTML
fig=go.Figure(data=[draw[idx]],
        layout=go.Layout(
            title='Probabilidad de que el cajero '+idx+' este desabastecido en la historia',
            titlefont=dict(family='Timer New Roman', size=20)
            )
        )        
HTML(fig.to_html())

In [ ]:
fig=go.Figure(data=[draw1[idx]],
        layout=go.Layout(
            title='Cantidad de efectivo el cajero '+idx+' historico',
            titlefont=dict(family='Timer New Roman', size=20)
            )
        )        
HTML(fig.to_html())

# Simulaciones

In [ ]:
from tqdm import tqdm
import pmdarima as pm
import datetime
from scipy import special
from scipy.stats import truncnorm
import numpy as np

#Funcion que extrae los outliers del los extremos ex1 y ex2
def filtroQuantil(DF,col=0,ex1=.005,ex2=.995):
    q=DF[col].quantile([.005, .995])
    return DF.loc[(DF[col]<=q.max())&(DF[col]>=q.min())]
    
simulaciones = gsheet2df('Atm Dvv 8-esp.csv')

simulaciones['FECHA']=pd.to_datetime(simulaciones['FECHA'], format='%d/%m/%Y')
start=True
# cantidad de muestras aleatorias
N=1000
# cantidad de dias a predecir
dias=10
#Cantidad maxima de dinero en un cajero automatico
dineroMaxATM=4.4*10**5
#extrae los 0.5% outliers del consumo
simulaciones["minCota"]=10**-9
simulaciones.SALDOINICIAL=simulaciones.SALDOINICIAL.str.replace('\.','')
simulaciones=simulaciones[simulaciones.CONSUMO.str.count('\.')==0].astype({'CONSUMO':'int64','SALDOINICIAL':'int64'})

'''
A=simulaciones.FECHA.drop_duplicates().to_frame()
for i in simulaciones["IDATM"].drop_duplicates():
  B=simulaciones.loc[simulaciones["IDATM"]==i,["FECHA","CONSUMO"]].rename(columns={"CONSUMO":i})
  A=A.merge(B,on="FECHA",how="left")
A=A.fillna(0)
#solo los que han consumido algun dia
CORR=A[A.columns[(A!=0).any(0)]].corr()
l=[]
for co in CORR.columns:
  a=CORR.loc[CORR[co]**2>.5,co].index.tolist()
  if a not in l:
    l+=[CORR.loc[CORR[co]**2>.5,co].index.tolist()]
'''

B=simulaciones[["IDATM","CONSUMO"]].groupby("IDATM").sum()
ATMS=list(set(politicas50.ATM.unique().tolist())-set(B.loc[B.CONSUMO==0].index.tolist()))
simulaciones=simulaciones.loc[simulaciones.IDATM.isin(ATMS)]

minConsumoLN=np.log1p(simulaciones.CONSUMO.min())
simulaciones=simulaciones.astype({"CONSUMO":'float64',"SALDOINICIAL":'float64'})
for atm in tqdm(ATMS):
    simulacionesATM=simulaciones.loc[(simulaciones.IDATM==atm)&(simulaciones.CONSUMO>=0)&(simulaciones.CONSUMO<=simulaciones.SALDOINICIAL),
                                     ["FECHA","CONSUMO","SALDOINICIAL","minCota"]].sort_values(by=['FECHA']).reset_index()
    hoy=simulacionesATM['FECHA'].max()
    simulacionesATM=simulacionesATM.reset_index(drop=True)

    '''
    SARIMAX:
    https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html?highlight=sarimax#statsmodels.tsa.statespace.sarimax.SARIMAX
    '''   
    #Modelo de estimacion y simulacion lognormal
    endog=np.log1p(simulacionesATM["CONSUMO"])
    # Fit your model
    model = pm.arima.auto_arima(endog,start_p=1, start_q=1, max_p=30, max_q=30,start_P=0,start_Q=0,max_P=30,max_D=15,max_Q=15, max_order=15, m=14,
                       seasonal=True,max_d=30,trace=True,n_jobs=-1,error_action='ignore',suppress_warnings=True,stepwise=True,random=True)

    # make your forecasts
    predict = model.predict(dias) # predict N steps into the future

    #model=sm.tsa.statespace.SARIMAX(endog,order=(4,1,3),seasonal_order=(4,0,3,14),enforce_invertibility=False,enforce_stationarity=False)
    #model_fit = model.fit(disp=False)
    #print("El MSE para el atm {0} es {1}".format(atm,((model_fit.predict(0,simulacionesATM.shape[0])-endog)**2).mean()**.5))
    #predict = model_fit.forecast(dias).values
    
    #Intervalo de confianza al erf(2**-.5)/2, para poder obtener la varianza de la normal rapidamente.
    #https://es.wikipedia.org/wiki/Distribuci%C3%B3n_normal
    sd=np.zeros(dias)
    for i in np.linspace(.01,.49,49):
        predict,predict_ci = model.predict(dias,return_conf_int=True,alpha=i)   
        sd+=((predict.reshape(-1,1)-predict_ci)**2).sum(1)/97
    sd=sd**.5
    a,b=(minConsumoLN-predict)*(predict>minConsumoLN)/sd,(np.log(dineroMaxATM)-predict)/sd
    SIM=pd.DataFrame({"atmID":[atm]*N})
    for i in range(dias):
        SIM["X"+str(1+i)]=np.expm1(truncnorm.rvs(a[i],b[i], loc=predict[i], scale=sd[i], size=N))
    if start:
        SIM1=SIM
        start=False
    else:
        SIM1=pd.concat([SIM1,SIM],axis=0)

Corelacion entre CONSUMO y SALDOINICIAL

In [ ]:
from google.colab import widgets,data_table
from IPython.display import display
tb = widgets.TabBar(ATMS)
for atm in ATMS:
  with tb.output_to(atm):
    display(data_table.DataTable(simulaciones.loc[(simulaciones.IDATM==atm)&(simulaciones.CONSUMO>=0)&(simulaciones.CONSUMO<=simulaciones.SALDOINICIAL),["CONSUMO","SALDOINICIAL"]].corr()))

In [ ]:
#body = dire+'atm_simulations_200_20200210.csv'
#SIM1 = pd.read_csv(body)
SIM1=SIM1.astype(float).astype({'atmID':'int64'})
data=[]
SIMatm=SIM1.loc[SIM1.atmID==int(idx),["X1","X2","X3","X4","X5","X6","X7","X8","X9","X10"]]
for k in ["X1","X2","X3","X4","X5","X6","X7","X8","X9","X10"]:
    data+=[go.Box(y=SIMatm[k],name=k)]
fig = go.Figure(data=data,
        layout=go.Layout(
            title='Boxplot de cada dia para las simulaciones del atm '+idx,
            titlefont=dict(family='Timer New Roman', size=20)
            ))    

HTML(fig.to_html())

Muestra las estadisticas de la simulacion del cajero.

In [ ]:
SIMatm.describe(percentiles=[.1,.25,.5,.75,.9])

Histograma de cada dia

In [ ]:
data=[]
for k in ["X1","X2","X3","X4","X5","X6","X7","X8","X9","X10"]:
    data+=[go.Histogram(x=SIMatm[k], name=k,opacity=0.75)]
fig = go.Figure(data=data,
        layout=go.Layout(
            title='Histograma de cada dia para las simulaciones del atm '+idx,
            titlefont=dict(family='Timer New Roman', size=20)
            ))
HTML(fig.to_html())

In [ ]:
data = [go.Histogram(x=SIMatm.values.reshape(SIMatm.size).tolist(),histnorm='probability',name="Histograma de la simulacion para el atm "+idx)]
fig = go.Figure(data=data,
        layout=go.Layout(
            title='Histograma total de las simulaciones del atm '+idx,
            titlefont=dict(family='Timer New Roman', size=20)
            ))
HTML(fig.to_html())

### Promedio de las diferencias entre las slimulaciones y las reales

In [ ]:
import datetime
historicos17Feb = gsheet2df('Atm Dvv 8-esp-17Feb.csv')

#body = dire+'atm_simulations_200_20200210.csv'
#SIM1 = pd.read_csv(body)
historicos17Feb=historicos17Feb[["FECHA","IDATM","CONSUMO"]].dropna()
historicos17Feb['FECHA']=pd.to_datetime(historicos17Feb['FECHA'], format='%d/%m/%Y')
historicos17Feb=historicos17Feb[historicos17Feb.FECHA>"2020-02-10"]
historicos17Feb['FECHA']=historicos17Feb['FECHA'].dt.strftime("%Y%m%d")


#body = dire+'Atm Dvv 8-esp.csv'
simulaciones = gsheet2df('Atm Dvv 8-esp.csv')
simulaciones['FECHA']=pd.to_datetime(simulaciones['FECHA'], format='%d/%m/%Y')
hoy=simulaciones.FECHA.max()

COLS=SIM1.columns[SIM1.columns.str.contains("X")]
SIM2=SIM1.rename(columns=dict(zip(SIM1[COLS],
(hoy+pd.to_timedelta(SIM1[COLS].columns.str.replace("X","").astype(int),unit='d')).strftime("%Y%m%d")))).rename(columns={"atmID":"IDATM"})
FECHA=historicos17Feb['FECHA'].drop_duplicates()
ATM=historicos17Feb['IDATM'].drop_duplicates()
Historicos17FEB=pd.DataFrame({'IDATM':ATM.tolist()})
for K in FECHA:
    Historicos17FEB[K]=historicos17Feb.loc[historicos17Feb.FECHA==K,["IDATM","CONSUMO"]].merge(historicos17Feb.IDATM,on="IDATM")["CONSUMO"]
Historicos17FEB=Historicos17FEB.dropna().astype(int)
SIM3=SIM2.merge(Historicos17FEB,on="IDATM")
fech=SIM3.columns[SIM3.columns.str.contains("_y")]
for d in fech:
    SIM3=SIM3.astype({d:'float64'})
    SIM3[d[:-1]+"diff"]=SIM3[d]-SIM3[d[:-1]+"x"]
SIM4=SIM3[["IDATM"]+SIM3.columns[SIM3.columns.str.contains("_diff")].tolist()]
SIM4.groupby(["IDATM"]).mean()

# Probabilidad de desabastecimiento del cajero

En el siguiente caso, se puede ver en el siguiente histograma, la probabilidad de los cajeros que no serán cargados.

In [ ]:
data = [go.Histogram(x=data0['probabilidad_stockout'],histnorm='probability')]
fig = go.Figure(data=data,
        layout=go.Layout(
            title='Histograma de probabilidad stockout',
            titlefont=dict(family='Timer New Roman', size=20)
            ))
HTML(fig.to_html())

En el siguiente caso, se puede ver en el siguiente histograma, la probabilidad de los cajeros que van a ser cargados.

In [ ]:
data = [go.Histogram(x=data1['probabilidad_stockout'],histnorm='probability')]
fig = go.Figure(data=data)
HTML(fig.to_html())

En la siguiente grafica, se pueden comparar ambos histogramas simultaneamente.

In [ ]:
from IPython.display import HTML
trace1 = go.Histogram(
    x=data0['probabilidad_stockout'],
    name='Cajeros sin recargar',
    opacity=0.75
)
trace2 = go.Histogram(
    x=data1['probabilidad_stockout'],
    name='Cajeros Cargados',
    opacity=0.75
)
data = [trace1, trace2]
fig = go.Figure(data=data)
HTML(fig.to_html())